In [97]:
%matplotlib inline
import sklearn as sk
import numpy as np
from numpy import random
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline 

import itertools
import math

# Bias Variance Decomposition

In this excercise we will experiment with the bias-variance decomposition we introduced in the first part of the course.

To do that, we will need to evaluate the average error a **single learning algorithm** commits on a **fixed example** when trained over different subsamples of a given dataset. To make things more interesting we will evaluate the average error over several examples and average  the results.

# Dataset

We will make use of a small synthetic dataset $D$.  Examples are pairs $(x,y) \in \mathbb{R}^2$ and the dataset is  a numpy array such that `D[i,:]` contains the $i^\text{th}$ example (consequently the array slice `D[:,0]`  contains all $x$s, while `D[:,1]` contains all the $y$s).

The dataset has been split into two parts, a training set named `Dtrain` and a test set named `Dtest`.

Let us begin by importing the training set and the test set:



In [98]:
from bias_var_data import Dtrain, Dtest

# Building the regressors

As mentioned above we will need to train a number of regressors over sub-samples of the training set. The following function will return a new dataset of the given `size` by subsampling from the given dataset `D`.

In [99]:
def subsample(D,size):
    Ds = D[random.random_integers(0,len(D)-1,size), :]
    return (Ds[:,0, np.newaxis], Ds[:,1, np.newaxis])

To build a regressor for one of the datasets, we shall fit a polynomial to it. The following function will fit a polynomial of the specified `degree` to the given dataset (here the dataset needs to be specified as a pair of `np.arrays`. `X` needs to contain the descriptions of the examples, `Y` needs to contain the corresponding labels. 

In [100]:
def build_regressor(X,Y,degree):
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(X, Y)
    return model

# Exercise

Consider the performances of the learning algorithm implemented in `build_regressor` as you vary the degree of the fitted polynomial from 0 to 9. Build 100 subsamples of `Dtrain` and perform the bias-variance error decomposition over the examples in `Dtest`. To best explain the details of the work you need to perform, the description of the work has been divided into three parts:

## First part

- Create 100 subsamples of size `15` of `Dtrain`;

In [101]:
subsamples = []
for i in range(100):
    subsamples.append(subsample(Dtrain, 15))
subsamples = np.array(subsamples)

/Users/gabriele/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: This function is deprecated. Please call randint(0, 99 + 1) instead
  from ipykernel import kernelapp as app


- for each degree `d` in $[0..9]$ and for each subsample `Dsub`:
    - fit a model of the degree `d` over the `Dsub`;
    - use the `.predict` method of the built model to make predictions for the examples in `Dtest`;
- collect the resulting data into a `np.array` and call it `results`;
    - if you do everything properly, the attribute `.shape` of `results` should return the tuple (10, 100, 30, 1), meaning that `results` should have:
        - one entry for each one of the 10 models;
        - each "model" entry, should have one entry for each one of the 100 subsamples;
        - each "subsample" entry should have one entry for each one of the 30 examples;
        - each "example" entry should have a single real value containing the prediction from the `.predict` method.
            
    - put in simpler terms, `result` should be such that `results[m,s,e,0]` returns the prediction of model `m` over example `e` of subsample `s`.

In [102]:
results = []
degrees = list(range(10))
for d in degrees:
    preds = []
    for sub in subsamples:
        model = build_regressor(sub[0],sub[1],d)
        pred = model.predict(Dtest[:,0].reshape(-1, 1))
        preds.append(pred)
    results.append(preds)
results = np.array(results)
print results.shape

(10, 100, 30, 1)


## Second part

Complete the following functions:

- `avg_error(` $\textbf{y}'$ `,` $y$ `)`: given a vector of predictions $\textbf{y}'$ and the correct prediction $y$ returns the average squared error committed by predictions in $\textbf{y}'$ w.r.t. $y$;
- `bias(` $\textbf{y}'$ `,` $y$ `)`: given the same parameters of `avg_error` returns the bias term of the squared error;
- `variance(` $\textbf{y}'$ `)`: given a vector of predictions $\textbf{y}'$ returns the variance term of the squared error;

In [103]:
def avg_error(y_, y):
    l = [(a-b)**2 for a,b in zip(y_, itertools.repeat(y))]
    return np.mean(l)

In [104]:
def bias(y_, y):
    e_y_ = np.mean(y_)
    return (y-e_y_)**2

In [105]:
def variance(y_):
    e_y_ = np.mean(y_)
    l = [(a-b)**2 for a,b in zip(y_, itertools.repeat(e_y_))]
    return np.mean(l)

The following two functions aggregate the values you computed (they will be useful in the final part of the exercise).

In [106]:
# Builds an array containing, for each model and each example, the bias term, the variance 
# term and the average error .

def BVEs(ys_, ys):
    return np.array([[bias( ys_[:,ex], ys[ex] ), 
                      variance( ys_[:,ex] ), 
                      avg_error( ys_[:,ex], ys[ex])] for ex in range(len(ys))])

In [107]:
# Computes the average over all examples of:
#    - the model's average squared error `e`
#    - the bias term `b` of the model's error
#    - the variance term `v` of the model's error
# and returns them as the tuple (b,v,e) 
def model_avg_bves(data, ys, model):
    bves = BVEs(data[model,:,:], ys)
    return [np.average(bves[:,0]), np.average(bves[:,1]), np.average(bves[:,2])]
    

## Third part

Let us now put in fruition our work by displaying, for each model, the three components of the error:

In [108]:
models_bves = np.array([model_avg_bves(results, Dtest[:,1], m) for m in range(len(degrees))])

print("%12s\t%20s\t%20s\t%20s") % ("Model degree", "Bias","Variance","Error")
print("%12s\t%20s\t%20s\t%20s") % ("-"*12,"-"*20,"-"*20,"-"*20)

for i,(b,v,e) in enumerate(models_bves):
    print("%12d\t%20.2f\t%20.2f\t%20.2f") % (i,b,v,e)

Model degree	                Bias	            Variance	               Error
------------	--------------------	--------------------	--------------------
           0	               54.44	                4.58	               59.02
           1	               53.50	               15.19	               68.69
           2	                6.40	                1.93	                8.33
           3	                6.39	                4.16	               10.55
           4	                6.11	               17.67	               23.78
           5	                9.70	              214.19	              223.89
           6	               21.20	             7813.64	             7834.84
           7	              384.95	           234962.13	           235347.07
           8	           239055.02	         12130877.43	         12369932.45
           9	           125009.13	        211730378.26	        211855387.39


Verify that indeed the sum of the bias and the variance terms equals the error

Assume you are devising a new system and that trying a new model is costly. You cannot affor to explore a large number of models. Describe how would you proceed to explore the space of the possible models basing your decisions on the bias/variance decomposition. You can use the above data as a running example.

I could increase the degree of polynomial until the error decreases.